# Fetch twitter text by API
This file is used to fetch twetter via Twitter API v2,

all tweets saved as a dict in tweets.npy

Author: Xiaofeng Zhang

Email: Xiaofzhang1@student.unimelb.edu.au

SID: 1194655

Reference: https://docs.tweepy.org/en/stable/client.html

In [ ]:
import numpy as np
import os
import tweepy
import json

In [ ]:
# get all tweets id from
def txt2set(filepath):
    _set = set()
    with open(filepath,"r") as f:
        ids=f.readline()
        while ids :
            ids = ids.replace("\n","")
            ids = ids.split(",")
            for id in ids:
                _set.add(id)
            ids = f.readline()
    return _set
test_set = txt2set("test.data.txt")
print(len(test_set))

train_set =  txt2set("train.data.txt")
print(len(train_set))
dev_set = txt2set("dev.data.txt")
print(len(dev_set))
tweet_set = train_set.union(test_set)
tweet_set = tweet_set.union(dev_set)

covid_set = txt2set("covid.data.txt")

# Initalize the npy file

In [ ]:
if os.path.exists("tweets.npy"):
    data_dict = np.load("tweets.npy",allow_pickle=True).item()
else:
    data_dict = {}
    np.save(data_dict)

for id in tweet_set:
    if id not in data_dict:
       data_dict[id] = None
#print(type(id))
np.save("tweets.npy",data_dict)

# Setting Twitter API

In [ ]:
bearer_token = "BearerToken"
train_data = "train.data.txt"
train_label = "train.label.txt"

client = tweepy.Client(bearer_token=bearer_token)


In [ ]:
data_dict = np.load("tweets.npy",allow_pickle=True).item()

#print(len(tweet_set))
tweet_list = list(tweet_set)
#print(len(tweet_list))
splited_list = []

dev_list = list(dev_set)
for i in range(0,len(dev_list),100):
    templist = dev_list[i:i+100]
    splited_list.append(templist)   
for i in range(len(splited_list)):
    ids = splited_list[i]
    head = ids[0]

    temp_t = [data_dict[id] for id in ids]
    if (any(t is None for t in temp_t) ):
       print(temp_t)

    try:
        print(i)
        Flag = False
        if (any(t is None for t in temp_t) ):
            Flag = True

        if Flag:
            tweets = client.get_tweets(ids=ids, tweet_fields=['context_annotations','created_at','geo'])
       
            for tweet in tweets.data:
                #print(tweet.id)
                data_dict[str(tweet.id)] = {'text':tweet.text,'created_at':tweet.created_at}
                #print(tweet.text)
                #print(tweet.created_at)
                #print('*'*10)
            #print(tweets.errors)
            for tweet in tweets.errors:
                data_dict[str(tweet['value'])] = tweet['title']
                #print(data_dict[str(tweet['value'])])
            print(i,"success")
        else:
            print(i,"passed")
            pass
    except Exception as e:
        print(i,e)
        
        np.save("tweets.npy",data_dict)
        break


#print(data_dict)

np.save("tweets.npy",data_dict)



# Check how much tweets are fetched

In [ ]:
test_dict = np.load("covid.npy",allow_pickle=True).item()
counter = 0
counter1 = 0
counter2 = 0

for t in covid_set:
    if test_dict[t] != None:
        if type(test_dict[t]) != type(''):
            #print(test_dict[t])
            counter +=1
        else :
            counter1 +=1
    else:
        counter2 +=1
print(counter,"successful")
print(counter1,"errors")
print(counter2,"has not get")